In [10]:

import numpy as np
import pandas as pd

import tensorflow as tf
import json
import pickle
import random
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d elvis23/mental-health-conversational-data

Dataset URL: https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data
License(s): copyright-authors
  0% 0.00/11.8k [00:00<?, ?B/s]
100% 11.8k/11.8k [00:00<00:00, 13.7MB/s]


In [4]:
# prompt: mount the drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:

import zipfile
zip = zipfile.ZipFile("/content/mental-health-conversational-data.zip",'r')
zip.extractall("/content")
zip.close()

In [11]:
import json

with open('/content/intents.json', 'r') as f:
    data = json.load(f)

df = pd.json_normalize(data['intents'])


In [12]:
pd.options.display.max_colwidth=None
df.head()

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, Hey there, Howdy, Hola, Bonjour, Konnichiwa, Guten tag, Ola]","[Hello there. Tell me how are you feeling today?, Hi there. What brings you here today?, Hi there. How are you feeling today?, Great to see you. How do you feel currently?, Hello there. Glad to see you're back. What's going on in your world right now?]"
1,morning,[Good morning],[Good morning. I hope you had a good night's sleep. How are you feeling today? ]
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night. Sweet dreams.]"


In [13]:
dic = {"tag":[], "patterns":[], "responses":[]}
for example in data['intents']:
    for pattern in example['patterns']:
        dic['patterns'].append(pattern)
        dic['tag'].append(example['tag'])
        dic['responses'].append(example['responses'])

In [15]:
# pd.options.display.max_rows=15
# pd.DataFrame([[labels,patterns]])

In [16]:
# pd.options.display.max_colwidth=15
df = pd.DataFrame.from_dict(dic)
df

,tag,patterns,responses
0,greeting,Hi,"[Hello there. Tell me how are you feeling today?, Hi there. What brings you here today?, Hi there. How are you feeling today?, Great to see you. How do you feel currently?, Hello there. Glad to see you're back. What's going on in your world right now?]"
1,greeting,Hey,"[Hello there. Tell me how are you feeling today?, Hi there. What brings you here today?, Hi there. How are you feeling today?, Great to see you. How do you feel currently?, Hello there. Glad to see you're back. What's going on in your world right now?]"
2,greeting,Is anyone there?,"[Hello there. Tell me how are you feeling today?, Hi there. What brings you here today?, Hi there. How are you feeling today?, Great to see you. How do you feel currently?, Hello there. Glad to see you're back. What's going on in your world right now?]"
3,greeting,Hi there,"[Hello there. Tell me how are you feeling today?, Hi there. What brings you here today?, Hi there. How are you feeling today?, Great to see you. How do you feel currently?, Hello there. Glad to see you're back. What's going on in your world right now?]"
4,greeting,Hello,"[Hello there. Tell me how are you feeling today?, Hi there. What brings you here today?, Hi there. How are you feeling today?, Great to see you. How do you feel currently?, Hello there. Glad to see you're back. What's going on in your world right now?]"
...,...,...,...
227,fact-29,How do I know if I'm unwell?,"[If your beliefs , thoughts , feelings or behaviours have a significant impact on your ability to function in what might be considered a normal or ordinary way, it would be important to seek help.]"
228,fact-30,How can I maintain social connections? What if I feel lonely?,"[A lot of people are alone right now, but we don't have to be lonely. We're all in this together. Think about the different ways to connect that are most meaningful for you. For example, you might prefer a video chat over a phone call, or you might prefer to text throughout the day rather than one set time for a video call. Then, work with your social networks to make a plan. You might video chat with your close friends in the evening and phone a family member once a week. Remember to be mindful of people who may not be online. Check in by phone and ask how you can help. The quality of your social connections matter. Mindlessly scrolling through social media and liking a few posts usually doesn't build strong social connections. Make sure you focus on strategies that actually make you feel included and connected. If your current strategies don't help you feel connected, problem-solve to see if you can find a solution. Everyone feels lonely at times. Maybe you recently moved to a new city, are changing your circle of friends, lost someone important in your life, or lost your job and also lost important social connections with coworkers. Other people may have physical connections to others but may feel like their emotional or social needs aren't met. Measures like social distancing or self-isolation can make loneliness feel worse no matter why you feel lonely now. Reach out to the connections you do have. Suggest ways to keep in touch and see if you can set a regular time to connect. People may hesitate to reach out for a lot of different reasons, so don't be afraid to be the one who asks. Look for local community support groups and mutual aid groups on social media. This pandemic is bringing everyone together, so look for opportunities to make new connections. These groups are a great way to share your skills and abilities or seek help and support. Look for specialized support groups. Support groups are moving online, and there are a lot of different support lines to call if you need to talk to someone.]"
229,fact-31,What's the difference between anxiety and stress?,"[Stress and anxiety are often used interchangeably, and there is overlap between stress and anxiety. Stress is related to the same fight, flight, or freeze response as anxi

In [17]:
df['tag'].unique()

array(['greeting', 'morning', 'afternoon', 'evening', 'night', 'goodbye',
       'thanks', 'no-response', 'neutral-response', 'about', 'skill',
       'creation', 'name', 'help', 'sad', 'stressed', 'worthless',
       'depressed', 'happy', 'casual', 'anxious', 'not-talking', 'sleep',
       'scared', 'death', 'understand', 'done', 'suicide', 'hate-you',
       'hate-me', 'default', 'jokes', 'repeat', 'wrong', 'stupid',
       'location', 'something-else', 'friends', 'ask', 'problem',
       'no-approach', 'learn-more', 'user-agree', 'meditation',
       'user-meditation', 'pandora-useful', 'user-advice',
       'learn-mental-health', 'mental-health-fact', 'fact-1', 'fact-2',
       'fact-3', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9',
       'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15',
       'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-20', 'fact-21',
       'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27',
       'fact-28', 'fact-29', '

In [18]:
df['tag'].nunique()

80

## Data Preprocessing¶


##### Now I am going to apply some text preprocessing stuffs such as lowering, punctuation removing and then tokenize the patterns.


In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
#  lists to be a token
tokenizer.fit_on_texts(df['patterns'])
# serialized JSON dictionary
tokenizer.get_config()

{'num_words': None,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': None,
 'document_count': 232,
 'word_counts': '{"hi": 2, "hey": 2, "is": 13, "anyone": 2, "there": 5, "hello": 1, "howdy": 1, "hola": 1, "bonjour": 1, "konnichiwa": 1, "guten": 1, "tag": 1, "ola": 1, "good": 5, "morning": 1, "afternoon": 1, "evening": 1, "night": 1, "bye": 3, "see": 2, "you": 38, "later": 1, "goodbye": 1, "au": 1, "revoir": 1, "sayonara": 1, "ok": 4, "then": 1, "fare": 1, "thee": 1, "well": 2, "thanks": 2, "thank": 3, "that\'s": 3, "helpful": 1, "for": 10, "the": 10, "help": 6, "than": 1, "very": 3, "much": 4, "nothing": 3, "who": 6, "are": 15, "what": 29, "tell": 6, "me": 19, "more": 6, "about": 20, "yourself": 3, "your": 3, "name": 4, "should": 3, "i": 95, "call": 1, "what\'s": 3, "can": 16, "do": 14, "created": 2, "how": 8, "were": 2, "made": 1, "my": 13, "am": 17, "go": 4, "by": 1, "could": 2, "give": 1, "a": 12, "hand": 1, "ple

In [20]:
vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)
vacab_size = 1000

number of unique words =  303


In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

X shape =  (232, 18)
y shape =  (232,)
num of classes =  80


In [22]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout

model = Sequential()
model.add(Input(shape=(X.shape[1])))
model.add(Embedding(input_dim=vacab_size+1, output_dim=100))
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32))
model.add(LayerNormalization())
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation="softmax"))
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 100)           100100    
                                                                 
 lstm (LSTM)                 (None, 18, 32)            17024     
                                                                 
 layer_normalization (Layer  (None, 18, 32)            64        
 Normalization)                                                  
                                                                 
 lstm_1 (LSTM)               (None, 18, 32)            8320      
                                                                 
 layer_normalization_1 (Lay  (None, 18, 32)            64        
 erNormalization)                                                
                                                                 
 lstm_2 (LSTM)               (None, 32)                8

# Split the data  

In [23]:
model_history = model.fit(x=X,
                          y=y,
                          batch_size=10,
                          callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=30)

Epoch 1/30
24/24 [==============================] - 9s 37ms/step - loss: 4.8584 - accuracy: 0.0172
Epoch 2/30
24/24 [==============================] - 1s 38ms/step - loss: 4.3420 - accuracy: 0.0517
Epoch 3/30
24/24 [==============================] - 1s 25ms/step - loss: 4.0794 - accuracy: 0.0776
Epoch 4/30
24/24 [==============================] - 1s 24ms/step - loss: 3.8416 - accuracy: 0.1034
Epoch 5/30
24/24 [==============================] - 1s 25ms/step - loss: 3.6024 - accuracy: 0.1164
Epoch 6/30
24/24 [==============================] - 1s 23ms/step - loss: 3.2639 - accuracy: 0.1810
Epoch 7/30
24/24 [==============================] - 1s 24ms/step - loss: 2.9423 - accuracy: 0.2716
Epoch 8/30
24/24 [==============================] - 1s 24ms/step - loss: 2.5663 - accuracy: 0.3319
Epoch 9/30
24/24 [==============================] - 1s 24ms/step - loss: 2.1725 - accuracy: 0.4181
Epoch 10/30
24/24 [==============================] - 1s 26ms/step - loss: 2.0152 - accuracy: 0.4397
Epoch 11/

In [27]:

model.save('/content/drive/MyDrive/ml/mental_health_chatbot.h5')

# Save the tokenizer
with open('/content/drive/MyDrive/ml/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Save the label encoder
with open('/content/drive/MyDrive/ml/label_encoder.pkl', 'wb') as f:
    pickle.dump(lbl_enc, f)


In [28]:

import pickle
import json
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained model and tokenizer
model = load_model('/content/drive/MyDrive/ml/mental_health_chatbot.h5')
tokenizer = pickle.load(open('/content/drive/MyDrive/ml/tokenizer.pkl', 'rb'))
lbl_enc = pickle.load(open('/content/drive/MyDrive/ml/label_encoder.pkl', 'rb'))

def chat():
    while True:
        # Get user input
        input_text = input("You: ")

        # Preprocess the input text
        input_text = input_text.lower()
        input_text = tokenizer.texts_to_sequences([input_text])
        input_text = pad_sequences(input_text, padding='post')

        # Predict the tag
        tag = lbl_enc.inverse_transform([np.argmax(model.predict(input_text))])[0]

        # Get the response based on the tag
        for intent in data['intents']:
            if intent['tag'] == tag:
                response = random.choice(intent['responses'])
                break

        # Print the response
        print("Bot:", response)

        # Check if the user wants to exit
        if input_text == 'quit':
            break

# Start the chat
chat()


You: hi
1/1 [==============================] - 2s 2s/step
Bot: Hi there. How are you feeling today?
You: just great
1/1 [==============================] - 2s 2s/step
Bot: I want to help you. I really do. But in order for me to help you, you're gonna have to talk to me.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [29]:
import pickle
import json
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random # Import the random module

# Load the trained model and tokenizer
model = load_model('/content/drive/MyDrive/ml/mental_health_chatbot.h5')
tokenizer = pickle.load(open('/content/drive/MyDrive/ml/tokenizer.pkl', 'rb'))
lbl_enc = pickle.load(open('/content/drive/MyDrive/ml/label_encoder.pkl', 'rb'))

def chat():
    while True:
        # Get user input
        input_text = input("You: ")

        # Check if the user wants to exit BEFORE preprocessing
        if input_text.lower() == 'quit':
            break

        # Preprocess the input text
        input_text = input_text.lower()
        input_text = tokenizer.texts_to_sequences([input_text])
        input_text = pad_sequences(input_text, padding='post')

        # Predict the tag
        tag = lbl_enc.inverse_transform([np.argmax(model.predict(input_text))])[0]

        # Get the response based on the tag
        for intent in data['intents']:
            if intent['tag'] == tag:
                response = random.choice(intent['responses'])
                break

        # Print the response
        print("Bot:", response)

# Start the chat
chat()

You: hi 
1/1 [==============================] - 2s 2s/step
Bot: Hi there. What brings you here today?
You: just great
1/1 [==============================] - 1s 1s/step
Bot: Talking about something really helps. If you're not ready to open up then that's ok. Just know that i'm here for you, whenever you need me.


KeyboardInterrupt: Interrupted by user

#  Model Testing


#### First we should apply some text preprocessing on the pattern that is passed to the function. Next we convert the text to vector of numbers and give it to model for prediction its tag; Finally based on the tag, we choose a answer(response) randomly and return it.



In [24]:
import re
import random

def generate_answer():
    while True:
        pattern=input()
        text = []
        txt = re.sub('[^a-zA-Z\']', ' ', pattern)
        txt = txt.lower()
        txt = txt.split()
        txt = " ".join(txt)
        text.append(txt)

        x_test = tokenizer.texts_to_sequences(text)
        x_test = np.array(x_test).squeeze()
        x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
        y_pred = model.predict(x_test)
        y_pred = y_pred.argmax()
        tag = lbl_enc.inverse_transform([y_pred])[0]
        responses = df[df['tag'] == tag]['responses'].values[0]
        print("model: {}".format(random.choice(responses)))
        if pattern=='quite':
            break;

In [25]:
generate_answer()

hello


ValueError: `sequences` must be a list of iterables. Found non-iterable: 150

In [ ]:
generate_answer()

 hi


ValueError: `sequences` must be a list of iterables. Found non-iterable: 95

In [ ]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import random
import pickle

with open("/kaggle/input/oumayma/oumayma.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        print("User: " ,end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print("ChatBot:" , np.random.choice(i['responses']))

print("Start messaging with the Chatbot (type quit to stop)!")

# call the function to start chat
# chat()